In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import matplotlib.pyplot as plt
from matplotlib import style
from torch.utils.data import DataLoader , TensorDataset, SubsetRandomSampler

In [2]:
MODEL_NAME = f"model-{time.asctime()}" 

In [3]:
BATCH_SIZE = 75
EPOCHS = 30
PT = "model/model2.pt" #path to existing model
IMG_SIZE = 50
VAL_PCT = 0.1
LR = 0.001
path = "sample_image.JPG" #path to single image
PATH = "training_data.npy" #path to training data


In [4]:
REBUILD_DATA =False # set to true to one once, then back to false unless you want to change something in your

In [5]:

# preparing data.
class DogsVSCats():
    #IMG_SIZE = 50
    CATS = "PetImages/Cat"
    DOGS = "PetImages/Dog"
    LABELS = {CATS: 0, DOGS: 1}
    training_data = []

    catcount = 0
    dogcount = 0

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                if "jpg" or "jpeg" in f:
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                        self.training_data.append([np.array(img), np.eye(2)[
                            self.LABELS[label]]])  # do something like print(np.eye(2)[1]), just makes one_hot
                        # print(np.eye(2)[self.LABELS[label]])

                        if label == self.CATS:
                            self.catcount += 1
                        elif label == self.DOGS:
                            self.dogcount += 1

                    except Exception as e:
                        pass
                        # print(label, f, str(e))

        np.random.shuffle(self.training_data)
        np.save(PATH, self.training_data)
        print('Cats:', dogsvcats.catcount)
        print('Dogs:', dogsvcats.dogcount)


In [6]:
if REBUILD_DATA:
    dogsvcats = DogsVSCats()
    dogsvcats.make_training_data()

In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [8]:
#neural network with dropout

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
#        self.bn1 = nn.BatchNorm2d(num_features=32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv2 = nn.Conv2d(32, 64, 5)
#        self.bn2 = nn.BatchNorm2d(num_features=64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.conv3 = nn.Conv2d(64, 128, 5)

        x = torch.randn(50, 50).view(-1, 1, 50, 50)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512)
        self.bn1 = nn.BatchNorm1d(num_features=512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.fc2 = nn.Linear(512, 2)
        self.dropout = nn.AlphaDropout(p=0.3)

    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))

        if self._to_linear is None:
            self._to_linear = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]
        return x

        '''>>> # With Learnable Parameters
        >>> m = nn.BatchNorm2d(100)
        >>> # Without Learnable Parameters
        >>> m = nn.BatchNorm2d(100, affine=False)
        >>> input = torch.randn(20, 100, 35, 45)
        >>> output = m(input)'''

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.fc2(x)
        x = self.dropout(x)
        return F.softmax(x, dim=1)

In [9]:
#creating model & parameters
net = Net().to(device)

#parameters
optimizer = optim.Adam(net.parameters(), lr=LR)
loss_function = nn.MSELoss()

In [14]:
#define a data loader function

def load_data(path):
    data = np.load(path, allow_pickle=True)
    print(len(data))   
    X = torch.Tensor([i[0] for i in data]).view(-1, 50, 50) #images
    X = X / 255.0 # scaling the data
    print(X.size())
    y = torch.Tensor([i[1] for i in data]) #observations / real labels
    
    return X, y

In [16]:
PATH = "testing_data.npy"

X, y = load_data(PATH)

24998
torch.Size([24998, 50, 50])


In [17]:
def create_sets(X,y):
    val_size = int(len(X) * VAL_PCT)
    print(val_size)

    #creating data and train sets.

    train_X = X[:-val_size]
    train_y = y[:-val_size]

    test_X = X[-val_size:]
    test_y = y[-val_size:]

    print(len(train_X))
    print(len(test_X))

    print(len(X)/2499)
    sampler = SubsetRandomSampler(list(range(int(len(train_X)))))

    test_sampler = SubsetRandomSampler(list(range(int(len(test_X)))))

    t_dataset = TensorDataset(train_X , train_y)
    trainloader = DataLoader(t_dataset , batch_size = BATCH_SIZE, sampler= sampler)

    v_dataset = TensorDataset(test_X, test_y)
    testloader = DataLoader(v_dataset, batch_size = BATCH_SIZE, sampler= test_sampler)
    
    return trainloader, testloader, val_size

train, test, validsize= create_sets(X,y)

n = len(X) - validsize # iteration number
print(n)

2499
22499
2499
10.003201280512204
22499


In [18]:
def Train(net):
    with open("model_new.log", "a") as f:
        for epoch in range(EPOCHS):
            for i in tqdm(range(0, n, BATCH_SIZE)):   
                dt = next(iter(train))
                batch_X = dt[0].view(-1, 1, 50, 50)
                batch_y = dt[1]
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)

                acc, loss = fwd_pass(batch_X, batch_y , train_=True)
                
                if i % 50 == 0:
                    val_acc, val_loss = Test(size=100)
                    f.write(
                        f"{MODEL_NAME},{round(time.time(), 3)},{round(float(acc), 2)},{round(float(loss), 4)},"
                        f"{round(float(val_acc), 2)},{round(float(val_loss), 4)},{epoch}\n")
                    torch.save({
                        'dropout cnn model': MODEL_NAME,
                        'epoch': epoch,
                        'model_state_dict': net.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict()
                    }, PT)


In [19]:
def Test(size=32):
    test_dt = next(iter(test))
    X, y = test_dt[0].view(-1, 1, 50, 50), test_dt[1]
    val_acc, val_loss = fwd_pass(X.view(-1, 1, 50, 50).to(device), y.to(device))
    return val_acc, val_loss

In [20]:
def create_acc_loss_graph(model_name):
    contents = open("model_new.log", "r").read().split("\n")

    times = []
    accuracies = []
    losses = []

    val_accs = []
    val_losses = []

    for c in contents:
        if model_name in c:
            name, timestamp, acc, loss, val_acc, val_loss, epoch = c.split(",")

            times.append(float(timestamp))
            accuracies.append(float(acc))
            losses.append(float(loss))

            val_accs.append(float(val_acc))
            val_losses.append(float(val_loss))

    fig = plt.figure()

    ax1 = plt.subplot2grid((2, 1), (0, 0))
    ax2 = plt.subplot2grid((2, 1), (1, 0), sharex=ax1)

    ax1.plot(times, accuracies, label="acc")
    ax1.plot(times, val_accs, label="val_acc")
    ax1.legend(loc=2)
    ax2.plot(times, losses, label="loss")
    ax2.plot(times, val_losses, label="val_loss")
    ax2.legend(loc=2)
    plt.show()

In [21]:
def fwd_pass(X, y, train_=False):
    if train_:
        net.zero_grad()
    outputs = net(X)
    matches = [torch.argmax(i) == torch.argmax(j) for i, j in zip(outputs, y)]
    acc = matches.count(True) / len(matches)
    loss = loss_function(outputs, y)

    if train_:
        loss.backward()
        optimizer.step()

    return acc, loss

In [22]:
style.use("ggplot")
MODEL_NAME = f"model-{time.asctime()}" 
print(MODEL_NAME)

model-Mon Jul 20 21:04:13 2020


In [ ]:
Train(net)
create_acc_loss_graph(MODEL_NAME)

 85%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                          | 255/300 [00:45<00:07,  5.83it/s]

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

In [ ]:
# loading existing model:

model = Net().to(device)
opt = optim.Adam(net.parameters(), lr=0.001)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
opt.load_state_dict(checkpoint['optimizer_state_dict'])

print("Optimizer's state_dict:")
for var_name in opt.state_dict():
    print(var_name, "\t", opt.state_dict()[var_name])

In [ ]:
def single_image_prediction(path)
    print(os.path.exists(path))

    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE )

    #if img == None: # older numpy / py2
        # fail !!
    #    print("fail!1")

    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    plt.imshow(img, cmap='gray')

    x = torch.Tensor([i for i in img]).view(-1, 1, 50, 50)
    x = x/255
    x = x.to(device)

    y= torch.Tensor([0,1]).to(device)

    model.eval()
    yhat = model(x)
    print(yhat)

    acc_, loss_ = fwd_pass(x, y)
    print(acc_, loss_)
    
    if acc_ == 1.0:
        print(f"Machine has predicted True! It predicted {yhat} and the input was a image of {y}. ")
    else: 
        print(f"Machine has predicted False! It predicted {yhat} and the input was a image of {y}. ")
    
    return

single_image_prediction(path)

In [ ]:
"""yhat = []
for i in range(500):
    fname = "D:/projects/Neural_Networks/coursera_v2/PetImages/dog/" + str(i)+ ".jpeg"
    #print(fname)
    img = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)
    
    #if img == None: # older numpy / py2
    # fail !!
    #    print("fail!1")
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    plt.imshow(img, cmap='gray')

    test_x = x.view(-1, 1, 50, 50)
    test_x = test_x.to(device)
    net.eval()
    #with torch.no_grad():
    yhat.append(net(test_x))
    print(yhat[i])
    
matches = [torch.argmax(yhat[i]) == torch.argmax(y) for i in range(len(yhat))]
acc = matches.count(True) / len(matches)
print(acc)"""

In [ ]:
"""sample = test_X[0:0 + BATCH_SIZE].view(-1, 1, 50, 50)
sample = sample.to(device)
#with torch.no_grad():
output = model(sample)
acc_, loss_ = fwd_pass(sample,test_y[0:0 + BATCH_SIZE].to(device) )
print(acc_, loss_)
print(output, "=?", test_y[0:0 + BATCH_SIZE]) """

In [ ]:
samples = np.load("t.npy", allow_pickle=True)
print(len(samples))

In [ ]:
x = torch.Tensor([i[0] for i in samples]).view(-1, 1, 50, 50)
x = x / 255.0 # scaling the data

print(x)
obs = torch.Tensor([i[1] for i in samples])

In [ ]:
model = Net().to(device)
opt = optim.Adam(net.parameters(), lr=0.001)

checkpoint = torch.load(PT)
model.load_state_dict(checkpoint['model_state_dict'])
opt.load_state_dict(checkpoint['optimizer_state_dict'])

model.eval()
outputs = model(x.to(device))

In [ ]:
print(outputs)

In [ ]:
acc_, loss_ = fwd_pass(x.to(device), obs.to(device))

In [ ]:
print(acc_, loss_)

In [ ]:
fname = "D:/projects/Neural_Networks/coursera_v2/PetImages/test/cat/cat.jpg"
    #print(fname)
img = cv2.imread(fname, cv2.IMREAD_GRAYSCALE)
    
    #if img == None: # older numpy / py2
    # fail !!
    #    print("fail!1")
img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
x_ = torch.Tensor([i for i in img]).view(-1, 1, 50, 50)
x_ = x_ / 255.0
print(x_)

outout= model(x_.to(device))

print(outout)

In [ ]:
len(train_X)

In [ ]:
len(X)

In [ ]:
val_size

In [ ]:
len(X)- val_size

In [ ]:
print(f"Machine has predicted True! It is a {outout}")